In [1]:
import glob
import math
import sys
import timeit

sys.path.insert(0, '../../')
from visualize_dataset import *
from losses import *
from metrics import *
from predict import *
from train import *
from utils import *

Using TensorFlow backend.


In [2]:
path_train_images = '../../../data/stanford_max_lesion_area_registered_fused_cropped/train/Slice/'
path_train_annotations = '../../../data/stanford_max_lesion_area_registered_fused_cropped/train/ROI/'
path_train_predictions = './predictions.50e/train/'

path_val_images = '../../../data/stanford_max_lesion_area_registered_fused_cropped/val/Slice/'
path_val_annotations = '../../../data/stanford_max_lesion_area_registered_fused_cropped/val/ROI/'
path_val_predictions = './predictions.50e/val/'

path_test_images = '../../../data/stanford_max_lesion_area_registered_fused_cropped/test/Slice/'
path_test_annotations = '../../../data/stanford_max_lesion_area_registered_fused_cropped/test/ROI/'
path_test_predictions = './predictions.50e/test/'

path_save_weights = './weights.50e/'
path_load_weights = None

train_vgg = False
path_vgg_weights = '../../../data/vgg16_weights_th_dim_ordering_th_kernels.h5'

n_classes = 2
input_height = 224
input_width = 224
n_channel_annotations = 1

In [3]:
print('Number train images: ', len(glob.glob(path_train_images + '*.png')))
print('Number val images: ', len(glob.glob(path_val_images + '*.png')))
print('Number test images: ', len(glob.glob(path_test_images + '*.png')))

('Number train images: ', 376)
('Number val images: ', 15)
('Number test images: ', 14)


In [4]:
model_name = 'vgg_segnet'
epochs = 50
train_batch_size = 8
val_batch_size = len(glob.glob(path_val_images + '*.png'))

In [5]:
optimizer = 'adadelta'
metrics = ['accuracy', dice, iou]

class_props = calc_class_proportions(path_train_annotations, n_classes, n_channel_annotations)
class_weights = calc_class_weights(path_train_annotations, n_classes, n_channel_annotations)
class_weights_log = calc_class_weights(path_train_annotations, n_classes, n_channel_annotations, scale = 'log')

print(class_props)
print(class_weights)
print(class_weights_log)

loss = weighted_categorical_crossentropy_loss(class_weights_log)

[0.97276311 0.02723689]
[ 1.         35.71490846]
[0.02761469 3.6031829 ]


In [ ]:
start = timeit.default_timer()
train(
    path_train_images, path_train_annotations, train_batch_size, 
    path_val_images, path_val_annotations, val_batch_size,
    n_classes, input_width, input_height, 
    model_name, epochs, loss, optimizer, metrics,
    path_save_weights, path_vgg_weights = path_vgg_weights, train_vgg = train_vgg, path_load_weights = None
)
end = timeit.default_timer()

('Model output shape', (None, 50176, 2))
('Epoch', 0)
Epoch 1/1
47/47 [==============================] - 1208s 26s/step - loss: 0.0445 - acc: 0.5913 - dice: 0.0881 - iou: 0.0463 - val_loss: 0.0516 - val_acc: 0.5603 - val_dice: 0.0568 - val_iou: 0.0292
('Epoch', 1)
Epoch 1/1
47/47 [==============================] - 1222s 26s/step - loss: 0.0271 - acc: 0.6762 - dice: 0.1189 - iou: 0.0636 - val_loss: 0.0515 - val_acc: 0.6628 - val_dice: 0.0632 - val_iou: 0.0326
('Epoch', 2)
Epoch 1/1
47/47 [==============================] - 1194s 25s/step - loss: 0.0217 - acc: 0.7954 - dice: 0.1762 - iou: 0.0974 - val_loss: 0.0520 - val_acc: 0.8180 - val_dice: 0.0938 - val_iou: 0.0492
('Epoch', 3)
Epoch 1/1
47/47 [==============================] - 1228s 26s/step - loss: 0.0185 - acc: 0.8585 - dice: 0.2393 - iou: 0.1377 - val_loss: 0.0604 - val_acc: 0.8861 - val_dice: 0.1018 - val_iou: 0.0536
('Epoch', 4)
Epoch 1/1
47/47 [==============================] - 1199s 26s/step - loss: 0.0154 - acc: 0.9131 - dice:

In [ ]:
print('Total training time: ', end - start)

In [6]:
epochs = 23

In [7]:
predict(
    path_train_images, path_train_annotations, path_train_predictions,
    n_classes, input_width, input_height, 
    model_name, epochs, loss, optimizer, metrics, 
    path_save_weights
)

              precision    recall  f1-score   support

           0       1.00      0.99      0.99  18452344
           1       0.61      0.99      0.75    413832

   micro avg       0.99      0.99      0.99  18866176
   macro avg       0.80      0.99      0.87  18866176
weighted avg       0.99      0.99      0.99  18866176

('F1/dice (binary): ', 0.7533684827182191)
('Jaccard/IOU: ', 0.6043233082706767)


In [ ]:
visualize_dataset(
    path_train_images, path_train_annotations, 
    n_classes, n_channel_annotations = n_channel_annotations,
    dir_predictions = path_train_predictions
)

In [9]:
predict(
    path_val_images, path_val_annotations, path_val_predictions,
    n_classes, input_width, input_height, 
    model_name, epochs, loss, optimizer, metrics,
    path_save_weights
)

              precision    recall  f1-score   support

           0       0.98      0.99      0.99    740323
           1       0.13      0.06      0.08     12317

   micro avg       0.98      0.98      0.98    752640
   macro avg       0.56      0.53      0.53    752640
weighted avg       0.97      0.98      0.97    752640

('F1/dice (binary): ', 0.08059337982951399)
('Jaccard/IOU: ', 0.04198869535125158)


In [ ]:
visualize_dataset(
    path_val_images, path_val_annotations, 
    n_classes, n_channel_annotations = n_channel_annotations,
    dir_predictions = path_val_predictions
)

In [11]:
predict(
    path_test_images, path_test_annotations, path_test_predictions,
    n_classes, input_width, input_height, 
    model_name, epochs, loss, optimizer, metrics,
    path_save_weights
)

              precision    recall  f1-score   support

           0       0.99      0.97      0.98    686380
           1       0.28      0.42      0.34     16084

   micro avg       0.96      0.96      0.96    702464
   macro avg       0.63      0.70      0.66    702464
weighted avg       0.97      0.96      0.97    702464

('F1/dice (binary): ', 0.33791338484856015)
('Jaccard/IOU: ', 0.20330672407091818)


In [ ]:
visualize_dataset(
    path_test_images, path_test_annotations, 
    n_classes, n_channel_annotations = n_channel_annotations, 
    dir_predictions = path_test_predictions
)